In [1]:
# normalize counts
# batch effect removal?
# cell type 
# annotate

In [4]:
4**8

65536

In [3]:
import scanpy as sc

ann = sc.read_h5ad("../../data/b13072bd-9cb6-42ca-9f4f-01252baef273.h5ad")
ann.shape

MemoryError: Unable to allocate 30.1 GiB for an array with shape (4038712131,) and data type int64